In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import math

from datetime import date, datetime, time, timedelta
from matplotlib import pyplot as plt
from pylab import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# https://zhuanlan.zhihu.com/p/56507515
# https://zhuanlan.zhihu.com/p/410655154
# https://zhuanlan.zhihu.com/p/21275919
# https://developer.aliyun.com/article/1056358

In [ ]:
src_path = 'indexProcessed.csv'
test_rate = 0.1               # proportion of dataset to be used as test set                   # proportion of dataset to be used as cross-validation set

test_num = 10                 # test number
                    

In [ ]:
df = pd.read_csv(src_path, sep = ",")
# df = df.drop(df.columns[[0]], axis = 1)
display(df)

In [ ]:
# Convert Date column to datetime
df.loc[:, 'Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')

# Change all column headings to be lower case, and remove spacing
df.columns = [str(x).lower().replace(' ', '_') for x in df.columns]

# Get month of each sample
# df['month'] = df['date'].dt.month

# # Sort by datetime
# df.sort_values(by='date', inplace=True, ascending=True)

df.head(10)

In [ ]:
index, times = print(df['index'][0]), 0
change_index = []
print(type(df))
for i in df['index']:
    if i != index:
        print(i, times)
        change_index.append(times)
        index = i
    times += 1
print(change_index, len(change_index))
# print(df[0: 2])
# print(df.iloc[0: 2])

In [ ]:
# print(df[0: 2])
# print(df.iloc[0: 2])
# print(df.loc[10000, 'index'])
# df.loc[df['index'] == 'HSI'] # copy dataframe with case
# print(df.iloc[len(df) - 1: len(df)]) # find last index
# print(df.adj_close.shift(-2))
df.head(8495)

In [ ]:
# ax = matplotlib.pyplot.subplots(4, 4)
rcParams['figure.figsize'] = 3, 3

index, times = 0, 0

while times < len(change_index):
        if times == 0:
                ax = df.iloc[: change_index[times + 1]].plot(x = 'date', y = 'adj_close', style = 'r-', grid = True)
                index = change_index[times]
                ax.set_xlabel(df['index'][change_index[times]])
                times += 1
                continue

        elif times == len(change_index) - 1:
                ax = df.iloc[change_index[times]:].plot(x = 'date', y = 'adj_close', style = 'r-', grid = True)
                ax.set_xlabel(df['index'][change_index[times]])
                break

        else:
                ax = df.iloc[index: change_index[times + 1]].plot(x = 'date', y = 'adj_close', style = 'r-', grid = True)
                index = change_index[times]
                ax.set_xlabel(df['index'][change_index[times]])
                times += 1


## data process function

In [ ]:
def wash_data(X, test_rate):
    num_test = int(test_rate * len(X))
    num_train = len(X) - num_test

    X_train, Y_train = X[:num_train], X[:num_train].adj_close
    X_test, Y_test = X[num_train:], X[num_train:].adj_close

    date_val = X_test['date']

    X_train = X_train[['high', 'low', 'close', 'volume']]
    X_test = X_test[['high', 'low', 'close', 'volume']]

    Y_train.shift(-1).fillna(Y_train.mean()) # make data to predict next day price

    X_train = X_train.shift(1).fillna(X_train.mean())

    X_test = X_test.shift(1).fillna(X_test.mean())

    Y_test = Y_test.shift(-1).fillna(Y_test.mean())

    return X_train, Y_train, X_test, Y_test, date_val

## Regression Switch Fuction (linear, polynomial and Losso)

In [ ]:
def regression_mode(X, test_rate, mode):

    from sklearn import linear_model
    from sklearn.preprocessing import PolynomialFeatures
    
    # print(X)
    X_train, Y_train, X_test, Y_test, date_val = wash_data(X, test_rate)

    if mode == 'linear':
        regression = linear_model.LinearRegression()
        regression.fit(X_train, Y_train)
        print(pd.DataFrame(regression.coef_.T, X_train.columns, columns = ['Coefficient']))
        print(regression.intercept_)
        Y_pred = regression.predict(X_test)
        print("Coefficients: \n", regression.coef_)

    elif mode == 'polynomial':
        regression = PolynomialFeatures(degree = 2)
        x_poly = regression.fit_transform(X_train)
        model = linear_model.LinearRegression()
        model.fit(x_poly, Y_train)
        Y_pred = model.predict(regression.fit_transform(X_test))

    else: 
        # Lasso
        # from sklearn.linear_model import Lasso, LassoCV
        regression = linear_model.Lasso(alpha = 5)
        regression.fit(X_train, Y_train)
        print(pd.DataFrame(regression.coef_.T, X_train.columns, columns = ['Coefficient']))
        print(regression.intercept_)
        Y_pred = regression.predict(X_test)
        print("Coefficients: \n", regression.coef_)
            

    from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, median_absolute_error
    # The mean squared error
    print("Mean squared error: %.2f" % mean_squared_error(Y_test, Y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print("Coefficient of determination: %.2f" % r2_score(Y_test, Y_pred))
    print("explained_variance_score: %.2f" % explained_variance_score(Y_test, Y_pred))
    mae = median_absolute_error(Y_test, Y_pred)
    print("Mae: %.2f" % mae)

    ax = matplotlib.pyplot.subplots(1, 1)
    rcParams['figure.figsize'] = 8,4
    ax = plt.scatter(date_val, Y_test, color="blue", marker = '.')
    bx = plt.plot(date_val, Y_pred, color="red")

## Main function

In [ ]:
index, times = 0, 0
model = 'linear' # polynomial or lasso

# regression_mode(df[:change_index[1]], test_rate, 'linear')

while times < len(change_index):
        print(times)
        if times == 0:
                regression_mode(df[:change_index[times + 1]], test_rate, model)
                index = change_index[times]
                times += 1
                continue

        elif times == len(change_index) - 1:
                regression_mode(df[change_index[times]:], test_rate, model)
                break

        else:
                regression_mode(df[index: change_index[times + 1]], test_rate, model)
                index = change_index[times]
                times += 1


## linear regression

In [ ]:
# # X_train = X_train.drop(index = 85883) # del extra data to predict the next day 
# # Y_train = Y_train.drop(index = 40920)
# # # X_test = X_test.drop(index = 54793)
# # # Y_test = Y_test.drop(index = 101091)

# date_val = X_test['date']

# # #del X_train['adj_close']

# X_train = X_train[['high', 'low', 'close', 'volume', 'closeusd']]
# X_test = X_test[['high', 'low', 'close', 'volume', 'closeusd']]
# # X_train = X_train[np.log10(['high', 'low', 'close', 'volume', 'closeusd'])]

# mean = Y_train.mean()
# print(mean)
# Y_train = Y_train.fillna(mean, inplace = False) 

# mean = X_train.mean()
# X_train = X_train.fillna(mean, inplace = False) 

# # mean = X_test.mean()
# # X_test = X_test.fillna(mean, inplace=False) 

# # mean = Y_test.mean()
# # Y_test = Y_test.fillna(mean, inplace=False) 

In [ ]:
# # from sklearn import linear_model
# # reg = linear_model.Ridge(alpha=.5)
# # # score = get_score(X_train, y_train)
# # reg.fit(X_train, Y_train)
# # reg.coef_
# # reg.intercept_

# from sklearn import linear_model
# regr = linear_model.LinearRegression()

# # Train the model using the training sets
# regr.fit(X_train, Y_train)

# print(pd.DataFrame(regr.coef_.T, X_train.columns, columns = ['Coefficient']))
# print(regr.intercept_)

# Y_pred = regr.predict(X_test)

In [ ]:
# from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, median_absolute_error

# # The coefficients
# print("Coefficients: \n", regr.coef_)
# # The mean squared error
# print("Mean squared error: %.2f" % mean_squared_error(Y_test, Y_pred))
# # The coefficient of determination: 1 is perfect prediction
# print("Coefficient of determination: %.2f" % r2_score(Y_test, Y_pred))
# print("explained_variance_score: %.2f" % explained_variance_score(Y_test, Y_pred))
# mae = median_absolute_error(Y_test, Y_pred)
# print("Mae: %.2f" % mae)

# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(regr, X_test, Y_test, cv=5)
# print(scores)

In [ ]:
# print(Y_pred)
# print(Y_test)
# ax = plt.scatter(date_val, Y_test, color="blue", marker = '+')
# bx = plt.scatter(date_val, Y_pred, color="red", marker = '.')
# plt.show()

In [ ]:
# times = 0
# diff = 0
# for i in Y_test:
#     if int(i) != int(Y_pred[times]):
#         print("different", i, Y_pred[times])
#         if (int(i) - int(Y_pred[times]) >= 1):
#             diff += 1
#             # print("different", i, Y_pred[times])
#     times += 1
# print(diff, "%.2f%%" % (diff / len(Y_pred)))

## Lasso

In [ ]:
# from sklearn.linear_model import Lasso, LassoCV
# lasso_model = Lasso(alpha = 5)
# lasso_model.fit(X_train, Y_train)

In [ ]:
# print(pd.DataFrame(lasso_model.coef_.T, X_train.columns, columns = ['Coefficient']))
# print(lasso_model.intercept_)

In [ ]:
# y_pred = lasso_model.predict(X_test)
# print(y_pred)
# print(Y_pred)

In [ ]:
# # The coefficients
# print("Coefficients: \n", regr.coef_)
# # The mean squared error
# print("Mean squared error: %.2f" % mean_squared_error(Y_test, y_pred))
# # The coefficient of determination: 1 is perfect prediction
# print("Coefficient of determination: %.2f" % r2_score(Y_test, y_pred))
# print("explained_variance_score: %.2f" % explained_variance_score(Y_test, y_pred))
# mae = median_absolute_error(Y_test, Y_pred)
# print("Mae: %.2f" % mae)

# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(regr, X_test, Y_test, cv=5)
# print(scores)

In [ ]:
# print(Y_pred)
# print(Y_test)
# ax = plt.scatter(date_val, Y_test, color="blue", marker = '+')
# bx = plt.scatter(date_val, y_pred, color="red", marker = '.')
# plt.show()

## Polynomial regression:

In [ ]:
# from sklearn.preprocessing import PolynomialFeatures

# poly_reg = PolynomialFeatures(degree = 2)

# x_poly = poly_reg.fit_transform(X_train)
# print(x_poly)
# model = linear_model.LinearRegression()
# model.fit(x_poly, Y_train)


In [ ]:
# # print(pd.DataFrame(model.coef_.T, X_train.columns, columns = ['Coefficient']))
# # print(pd.DataFrame(model.coef_.T))
# print(model.intercept_)

In [ ]:
# y_pred_ph = model.predict(poly_reg.fit_transform(X_test))
# print(y_pred_ph)
# print(Y_pred)

In [ ]:
# # The coefficients
# print("Coefficients: \n", regr.coef_)
# # The mean squared error
# print("Mean squared error: %.2f" % mean_squared_error(Y_test, y_pred_ph))
# # The coefficient of determination: 1 is perfect prediction
# print("Coefficient of determination: %.2f" % r2_score(Y_test, y_pred_ph))
# print("explained_variance_score: %.2f" % explained_variance_score(Y_test, y_pred_ph))
# mae = median_absolute_error(Y_test, y_pred_ph)
# print("Mae: %.2f" % mae)

In [ ]:
# print(y_pred_ph)
# print(Y_test)
# ax = plt.scatter(date_val, Y_test, color="blue", marker = '+')
# bx = plt.scatter(date_val, y_pred_ph, color="red", marker = '.')
# plt.show()